# Zero Flowrate Degenerate Example

with three separation units

Created by Alex Dowling (alexdowling.net) at the University of Wisconsin-Madison

Load modeling package (JuMP) and optimization solver (IPOPT)

In [ ]:
using JuMP
using Ipopt

# Define Problem

Declare streams, components (chemical species), and unites

In [ ]:
# Streams
S = 2:9

# Components (chemical species)
C = ["A", "B"]

# Units
U = ["U1", "U2", "U3"]

Specify flowrate connectivity using dictionaries

In [ ]:
inlets = Dict{ASCIIString,Integer}("U1"=>2, "U2"=>5, "U3"=>4)
outletV = Dict{ASCIIString,Integer}("U1"=>3, "U2"=>6, "U3"=>8)
outletL = Dict{ASCIIString,Integer}("U1"=>4, "U2"=>7, "U3"=>9)

Specify feed streams and total feed flowrate

In [ ]:
feeds = [2, 5]
feedflow = Dict{AbstractString, Float64}("A"=>0.55, "B"=>0.45)

Define costs for operating each unit

In [ ]:
ecost = Dict{AbstractString,Float64}("U1"=>1.5, "U2"=>1.0)

Specify equilibrium partition coefficient for each unit

In [ ]:
K = Dict{Tuple{ASCIIString, ASCIIString}, Float64}(
		("U1","A")=>1.008,		("U1","B")=>0.9,
		("U2","A")=>1.099,		("U2","B")=>0.9,
		("U3","A")=>1.093,		("U3","B")=>0.9	)

# Build Optimization Model

In [ ]:
m = Model(solver=IpoptSolver())

## General flowsheet model

In [ ]:
# Total molar flowrate
@variable(m, f[S] >= 0)

# Component molar flowrate
@variable(m, fc[S,C] >= 0)

# Component mole fraction
@variable(m, 0.001 <= x[S,C] <= 1)

# Unit model
for u in U
	
	i = inlets[u]
	v = outletV[u]
	l = outletL[u]
	
	# Overall mass balance
	@constraint(m, f[i] == f[v] + f[l])

	for c in C
		# Component mass balance
		@constraint(m, fc[i,c] == fc[v,c] + fc[l,c])
	
		# vapor-liquid equilibrium
		@constraint(m, x[v, c] == K[(u,c)]*x[l,c])
	
	end
	
	# Unit summation (Rachford-Rice equation)
	# This constraint is redundant!!!
	# @constraint(m, sum{x[v,c] - x[l,c], c in C} == 0)
	
end

# Stream model: mole fraction specification
for s in S
	for c in C
		@constraint(m, fc[s,c] == f[s]*x[s,c])
	end
end

## Additional specifications

In [ ]:
@variable(m, 0.551 <= purityA <= 1.0)
@variable(m, 0.9 <= recoveryA <= 1.0)

# Equipment cost
ecost = Dict{AbstractString,Float64}("U1"=>1.5, "U2"=>1.0, "U3"=>0.5)

# Set total feed to one
@constraint(m, sum{f[s], s in feeds} == 1)

# Calculate purity of A
@constraint(m, purityA*sum{sum{f[s], s in outletV[u]}, u in U} == sum{sum{fc[s,"A"], s in outletV[u]}, u in U})

# Calculate recovery of A
@constraint(m, recoveryA == sum{sum{fc[s,"A"], s in outletV[u]}, u in U} / feedflow["A"])

# Objective
@objective(m, Min, sum{sum{f[s], s in inlets[u]}*ecost[u], u in U} - 100*purityA)

for s in feeds
	for c in C
		setupperbound(x[s,c], feedflow[c])
		setlowerbound(x[s,c], feedflow[c])
	end
end

## Initialization

Strategy: Set all stream total molar flowrates to unity and assume composition matches feed

In [ ]:
for s in S
	
	setvalue(f[s], 1.0)
	
	for c in C
		setvalue(fc[s,c], feedflow[c])
		setvalue(x[s,c], feedflow[c])
	end
end

## Print Model

In [ ]:
m

# Solve Model

In [ ]:
solve(m)

Examine solution

In [ ]:
for s in S
    print("Stream ",s,"\t")
    @printf "f = %.4f \t" getvalue(f[s])
    for c in C
        @printf "x_%s = %.4f \t" c getvalue(x[s,c]) 
    end
    println(" ")
end

# Run Degeneracy Hunter

In [ ]:
include("../DegeneracyHunter.jl")

In [ ]:
DegeneracyHunter.degeneracyHunter(m);